### Transform data in given df column(s)

In [1]:
import pandas as pd
from pandas.core.strings import StringMethods

InputDataSet = [
	{
		"name": "Reece Bartlett",
		"company": "Vivamus Rhoncus LLC",
		"country": "Spain"
	},
	{
		"name": "Finn Bryant",
		"company": "Nunc Risus Company",
		"country": "Australia"
	},
	{
		"name": "Kay Walter",
		"company": "Rhoncus Id Incorporated",
		"country": "Philippines"
	},
	{
		"name": "Ann Miranda",
		"company": "Quisque Tincidunt Pede Limited",
		"country": "Spain"
	},
	{
		"name": "Kylie Glenn",
		"company": "Dignissim Maecenas Company",
		"country": "Belgium"
	}
]

df = pd.DataFrame.from_records(InputDataSet)

# Function to transform string data in given df column(s)
def tranform_df_col(df, transform):
	return df.apply(lambda x: transform(x.str) if x.dtype == "object" else x)

# Get data from input query
df[['name', 'company']] = tranform_df_col(df[['name', 'company']], StringMethods.lower)

OutputDataSet = df

display(OutputDataSet)

,name,company,country
0,reece bartlett,vivamus rhoncus llc,Spain
1,finn bryant,nunc risus company,Australia
2,kay walter,rhoncus id incorporated,Philippines
3,ann miranda,quisque tincidunt pede limited,Spain
4,kylie glenn,dignissim maecenas company,Belgium


### Same as above, but in SQL Server, using `sp_execute_external_script`

In [1]:
-- random data
DROP TABLE IF EXISTS #TEMP_VALS
SELECT * INTO
	#TEMP_VALS
FROM
	(VALUES
		('Reece Bartlett', 'Vivamus Rhoncus LLC', 'Spain'),
		('Finn Bryant', 'Nunc Risus Company', 'Australia'),
		('Kay Walter', 'Rhoncus Id Incorporated', 'Philippines'),
		('Ann Miranda', 'Quisque Tincidunt Pede Limited', 'Spain'),
		('Kylie Glenn', 'Dignissim Maecenas Company', 'Belgium')
	) vals ([name], [company], [country])

-- the input query
DECLARE @input_query NVARCHAR(MAX) = N'SELECT * FROM #TEMP_VALS'

-- execute the script
EXEC sp_execute_external_script
@language = N'Python',
@script = N'
import pandas as pd
from pandas.core.strings import StringMethods

# Function to transform string data in given df column(s)
def tranform_df_col(df, transform):
	return df.apply(lambda x: transform(x.str) if x.dtype == "object" else x)

df = InputDataSet

# Get data from input query
df[[''name'', ''company'']] = tranform_df_col(df[[''name'', ''company'']], StringMethods.lower)

OutputDataSet = df
',
@input_data_1 = @input_query
WITH RESULT SETS
((
	[name] varchar(100),
	[company] varchar(100),
	[country] varchar(100)
))

(5 rows affected)

Total execution time: 00:00:00.360

name,company,country
reece bartlett,vivamus rhoncus llc,Spain
finn bryant,nunc risus company,Australia
kay walter,rhoncus id incorporated,Philippines
ann miranda,quisque tincidunt pede limited,Spain
kylie glenn,dignissim maecenas company,Belgium
